# Linkování C kódu a C knihoven s Pythonem

V této části si ukážeme, jak lze v Pythonu používat C knihovny a kód. 

- Využijeme knihovnu `ctypes` případně `cffi` (pro načítání knihoven) 
- Nativní Pythonovské rozhraní pro C, případně `cython`

## Nejprve C kód
Máme jednoduchý C kód, který provádí násobení matice a vektoru. 

Krátké opakování C:
- `*` před proměnnou znamená, že se jedná o ukazatel
- pole se v C zapisuje jako ukazatel na první prvek, sami o sobě neobsahují informaci o délce a tvaru
- `void` znamená, že funkce nic nevrací


In [ ]:
%%writefile matrix_vector_multiply.c

void matrix_vector_multiply(float *matrix, float *vector, float *result, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        result[i] = 0;
        for (int j = 0; j < cols; j++) {
            result[i] += matrix[i * cols + j] * vector[j];
        }
    }
}


#### Kompilejme C kód
Pro uživatele linuxu je tento krok velmi jednoduchý, neboť všechny potřebné komponenty již máte v systému.

Pokud se Vám pod Windows podaří zkompilovat C kód (ať už Microsoft Visual Studio C++ compiler (MSVC) nebo MinGW-w64) bude vašim výstupem `.dll` knihovna, tak pak pozor při načítání. V tomto případě bude potřeba použít `WinDLL` místo `CDLL`.

In [ ]:
!gcc -shared -fPIC -o libmatrix_vector_multiply.so matrix_vector_multiply.c

# Načtení C shared knihovny (`.so`) pomocí Ctypes (uvnitř NumPy)

Moduly Ctypes a Numpy lze použít pro snadnou integraci mezi jazyky Python a C.

- `Ctypes` je modul Pythonu, který poskytuje kompatibilní datové typy pro C a Python, a umožňuje volat funkce z dynamicky sdílených knihoven napsaných v jazyce C.
- `Numpy` obsahuje integraci `Ctypes` - lze tedy použít čistě Ctypes, případně Ctypes podmodul v Numpy


Pro nalinkování předchozí funkce `matrix_vector_multiply` z C knihovny `libmatrix_vector_multiply.so` je třeba:


1. Načítání dynamicky sdílené knihovny: Kód načítá dynamicky sdílenou knihovnu (také známou jako "shared library" nebo ".so" soubor), která obsahuje funkce a datové struktury napsané v jazyce C. Toto se provádí pomocí `ctypes.CDLL` nebo `np.ctypeslib.load_library`.
2. Přistoupení k oběktu reprezentujícímu C funkci (používá se `.` syntaxe, jako u modulů)
3. Mapování C datových typů na Python datové typy: Kód mapuje C datové typy na jejich odpovídající Python datové typy pomocí ctypes modulu.  Definuje se atribut `argtypes` pro každou C funkci, která se volá, a `restype` pro funkce, které vrací hodnotu.


Více informací o Ctypes naleznete v [dokumentaci](https://docs.python.org/3/library/ctypes.html), nebo v NumPy [dokumentaci](https://numpy.org/doc/stable/reference/routines.ctypeslib.html).

In [ ]:
import numpy as np

# Load the shared library
lib_matrix_vector_multiply = np.ctypeslib.load_library(libname='libmatrix_vector_multiply.so', loader_path='.')

# Define the matrix_vector_multiply function from the shared library
matrix_vector_multiply = lib_matrix_vector_multiply.matrix_vector_multiply
matrix_vector_multiply.argtypes = [np.ctypeslib.ndpointer(dtype=np.float32),
                                   np.ctypeslib.ndpointer(dtype=np.float32),
                                   np.ctypeslib.ndpointer(dtype=np.float32),
                                   np.ctypeslib.c_intp,
                                   np.ctypeslib.c_intp]

matrix = np.array([[1, 2, 3],
                    [4, 5, 6]], dtype=np.float32)

vector = np.array([1, 2, 3], dtype=np.float32)
result = np.zeros((2), dtype=np.float32)

matrix_vector_multiply(matrix, vector, result, matrix.shape[0], matrix.shape[1])

print("Result vector:")
print(result)


In [ ]:
matrix@vector

Totéž přímo pomocí Ctypes:

In [ ]:
import ctypes
import numpy as np

# Načtěte sdílenou knihovnu
lib_matrix_vector_multiply = ctypes.CDLL('./libmatrix_vector_multiply.so')

# Definujte funkci matrix_vector_multiply ze sdílené knihovny
matrix_vector_multiply = lib_matrix_vector_multiply.matrix_vector_multiply
matrix_vector_multiply.argtypes = [ctypes.POINTER(ctypes.c_float),
                                   ctypes.POINTER(ctypes.c_float),
                                   ctypes.POINTER(ctypes.c_float),
                                   ctypes.c_int,
                                   ctypes.c_int]

matrix = np.array([[1, 2, 3],
                    [4, 5, 6]], dtype=np.float32)

vector = np.array([1, 2, 3], dtype=np.float32)
result = np.zeros((2), dtype=np.float32)

# Konverze numpy polí na C pole
matrix_ptr = matrix.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
vector_ptr = vector.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
result_ptr = result.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

matrix_vector_multiply(matrix_ptr, vector_ptr, result_ptr, matrix.shape[0], matrix.shape[1])

print("Result vector:")
print(result)


### Takto to dělá CFFI

Jednou z výhod CFFI je možnost automatického načtení typů z hlavičnového souboru.

In [ ]:
%%writefile matrix_vector_multiply.h

void matrix_vector_multiply(float *matrix, float *vector, float *result, int rows, int cols);

In [ ]:
#!pip install cffi

In [ ]:
import numpy as np
from cffi import FFI

ffi = FFI()

# Načtěte obsah hlavičkového souboru
with open('matrix_vector_multiply.h', 'r') as f:
    header = f.read()

# Zadejte hlavičkový soubor s deklarací funkce
ffi.cdef(header)

# Načtěte sdílenou knihovnu
lib_matrix_vector_multiply = ffi.dlopen('./libmatrix_vector_multiply.so')

matrix = np.array([[1, 2, 3],
                   [4, 5, 6]], dtype=np.float32)

vector = np.array([1, 2, 3], dtype=np.float32)
result = np.zeros((2), dtype=np.float32)

# Konverze numpy polí na C pole
matrix_ptr = ffi.cast("float *", matrix.ctypes.data)
vector_ptr = ffi.cast("float *", vector.ctypes.data)
result_ptr = ffi.cast("float *", result.ctypes.data)

lib_matrix_vector_multiply.matrix_vector_multiply(matrix_ptr, vector_ptr, result_ptr, matrix.shape[0], matrix.shape[1])

print("Result vector:")
print(result)

In [ ]:
matrix@vector

# Vytváření Python rozharaní pro C kód v Pythonu
Python poskytuje hlavičkové soubory pro C, které umožňují přistupovat k Python objektům. 
Pomocí těchto hlavičkových souborů lze vytvářet C knihovnu, které se pro Python bude tvářit jako nativní Python knihovna.

Ukážeme si malou ukázku, více v dokumentaci [zde](https://docs.python.org/3/c-api/).

### Parsování vstupu z Pythonu do C

Funkce která bude volána z Pythonu bude v C vypadat nějak takto:

```c
static PyObject* moje_fce(PyObject *self, PyObject *args) 
```

Tedy všechny vstupy z Pythonu budou předány jako `PyObject *args`. A očekává se návratová hodnota `PyObject*`.

Proto prvním hlavním stavebním kamenem C API je parsování vstupu z Pythonu do C.

Funkce `PyArg_ParseTuple` je součástí Python C API a slouží k parsování argumentů předaných z Pythonu do funkce v jazyce C. Umožňuje konverzi Python objektů předaných jako argumenty do vaší C funkce na hodnoty v jazyce C.

Signatura funkce je následující:
`int PyArg_ParseTuple(PyObject *args, const char *format, ...);`

**Parametry**

1.  `args`: Ukazatel na Python tuple objekt obsahující argumenty předané z Pythonu. (Tedy to co přišlo jako `*args`)
2.  `format`: Formátovací řetězec, který specifikuje očekávané typy a počet argumentů v tuple. Každý znak ve formátovacím řetězci odpovídá očekávanému typu argumentu.
    *   `s`: Řetězec (ukončený nulou)
    *   `i`: Celé číslo
    *   `f`: Číslo s plovoucí řádovou čárkou
    *   `d`: Dvojitá přesnost (double)
    *   `O`: Python objekt (obecný)
    *   `|`: Udává, že následující argumenty jsou volitelné
    *   `O!`: Python objekt specifického typu, následovaný ukazatelem na Python typový objekt
3.  `...`: Proměnný počet ukazatelů na proměnné v jazyce C, které budou obsahovat převedené hodnoty argumentů. Tyto ukazatele by měly odpovídat počtu a pořadí specifikátorů formátu ve formátovacím řetězci.



### Vytváření obektů/struktur viditelných pro Python
#### Metody
`PyMethodDef` je struktura v Python C API, která definuje metody modulu, obsahuje následující záznamy:

1. Název metody jako řetězec (např. "my_c_function").
2. Ukazatel na C funkci, která implementuje metodu (např. `my_c_function`).
3. Příznak způsobu předání argumentů (`METH_VARARGS` pro proměnný počet argumentů ve formě n-tice).
4. Řetězec s dokumentací metody ("Multiply all elements of a NumPy array by 2").

Seznam metod je pak pole těchto struktur, které uvádí metody, které budou v modulu dostupné. 

Poslední prvek pole musí obsahovat nuly a NULL ukazatele, které slouží jako ukončovací značka pro pole metod.


```c
static PyMethodDef MyCModuleMethods[] = {
    {"my_c_function", my_c_function, METH_VARARGS, "Multiply all elements of a NumPy array by 2"},
    {NULL, NULL, 0, NULL}
};
```

Při vytváření C modulu pro Python je toto pole metod předáno do funkce `PyModule_Create`, která vytvoří modul s definovanými metodami.


#### Modul
Informace o modulu definuje struktura `PyModuleDef` z Python C API. Skládá se z:

1. `PyModuleDef_HEAD_INIT`: Makro pro inicializaci hlavičky struktury modulu.
2. Název modulu jako řetězec (např. "my_c_module").
3. Ukazatel na řetězec s dokumentací modulu (v tomto případě `NULL`, protože dokumentace není poskytnuta).
4. Velikost prostoru modulu (`-1` značí, že modul nemá žádný stav a všechny jeho funkce jsou globální).
5. Ukazatel na pole metod definovaných v modulu (viz stuktura `PyMethodDef`).


```c
static struct PyModuleDef my_c_module = {
    PyModuleDef_HEAD_INIT,
    "my_c_module",
    NULL,
    -1,
    MyCModuleMethods
};
```

Tato struktura modulu je poté předána do funkce `PyModule_Create`, která vytvoří modul s definovanými metodami a případným stavem.

#### Inicializace modulu
Pro nás poslední "povinnou" částí je inicializace modulu ať skutečně vše co jsme doposud definovali (zatím jenom struktury v C) je viditelné pro Python.

Funkce `PyMODINIT_FUNC PyInit_my_c_module(void)` je inicializační funkce modulu, která se volá, když Python načítá váš C modul. Tato funkce má následující úkoly:

1. Vytvořit modul pomocí funkce `PyModule_Create`, která přijímá ukazatel na strukturu `PyModuleDef` (v tomto případě `&my_c_module`).
2. Zkontrolovat, zda se modul úspěšně vytvořil. Pokud `module` je `NULL`, vrátit `NULL` a ukončit inicializaci.
3. Inicializovat externí závislosti, pokud existují. V tomto příkladu se volá funkce `import_array()` pro inicializaci NumPy C API.
4. Vrátit vytvořený modul, který bude následně zaregistrován v Pythonu.

```c
PyMODINIT_FUNC PyInit_my_c_module(void) {
    PyObject* module = PyModule_Create(&my_c_module);
    if (module == NULL) {
        return NULL;
    }

    // Initialize the NumPy C API
    import_array();

    return module;
}
```

Tato inicializační funkce musí vracet `PyMODINIT_FUNC` a musí mít název `PyInit_` následovaný názvem modulu.

In [ ]:
%%writefile mvc_mylib.c

#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION

#include <Python.h>
#include <numpy/arrayobject.h>
#include "matrix_vector_multiply.c"

static PyObject* mvp_func_py(PyObject *self, PyObject *args) {
    // Declare variables
    PyArrayObject *matrix, *vector, *result;
    int rows, cols;

    if (!PyArg_ParseTuple(args, "O!O!O!", &PyArray_Type, &matrix, &PyArray_Type, &vector, &PyArray_Type, &result)) {
        return NULL;
    }

    rows = (int)PyArray_DIM(matrix, 0);
    cols = (int)PyArray_DIM(matrix, 1);

    matrix_vector_multiply((float *)PyArray_DATA(matrix), (float *)PyArray_DATA(vector), (float *)PyArray_DATA(result), rows, cols);

    Py_RETURN_NONE;
};

static PyMethodDef MatrixVectorMultiplyMethods[] = {
    {"mvp_func_py", mvp_func_py, METH_VARARGS, "Perform matrix-vector multiplication."},
    {NULL, NULL, 0, NULL}
};

static struct PyModuleDef matrix_vector_multiply_module = {
    PyModuleDef_HEAD_INIT,
    "mvc_mylib",
    NULL,
    -1,
    MatrixVectorMultiplyMethods
};

PyMODINIT_FUNC PyInit_mvc_mylib(void) {
    PyObject* module = PyModule_Create(&matrix_vector_multiply_module);
    import_array();
    return module;
};

In [ ]:
%%writefile setup.py
from setuptools import setup, Extension
import numpy as np

matrix_vector_multiply_module = Extension(
    'mvc_mylib',
    sources=['mvc_mylib.c'],
    include_dirs=[np.get_include()],
)

setup(
    name='mvc_mylib',
    version='1.0',
    description='A C extension for multiplying a matrix and a vector',
    ext_modules=[matrix_vector_multiply_module],
)


In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import numpy as np
import mvc_mylib


In [ ]:
matrix = np.array([[1, 2, 3, 4],
                   [5, 6, 7, 8],
                   [9, 10, 11, 12]], dtype=np.float32)

vector = np.array([2, 1, 3, 0], dtype=np.float32)
result = np.zeros((3), dtype=np.float32)

mvc_mylib.mvp_func_py(matrix, vector, result)

print("Result:")
print(result)

In [ ]:
matrix@vector

## Totéž pomocí Cythonu
K načtení C filu z Cythonu budeme potřebovat následující příkazy:

- `cdef extern from "matrix_vector_multiply.c"`: Tento příkaz importuje externí C funkci `matrix_vector_multiply` z hlavičkového souboru "matrix_vector_multiply.c". Tímto způsobem lze v Cythonu volat funkce napsané v jazyce C.
ozhraní mezi rychlými C funkcemi a Python kódem.
- `cnp.ndarray[cnp.float32_t, ndim=2]`, `cnp.ndarray[cnp.float32_t, ndim=1]`: Tyto typy definují NumPy pole s konkrétním datovým typem a počtem dimenzí. V tomto případě se jedná o pole s jednoduchou přesností plovoucích čísel (float32) a jednorozměrné nebo dvourozměrné pole.
- `<float*> &matrix[0, 0], <float*> &vector[0], <float*> &result[0]`: Tyto konstrukce provádějí type casting ukazatelů na datový typ `float*`. Díky tomu je možné předat adresy prvků NumPy polí do C funkce.

Pro kompilaci je třeba dodat cestu odkud se mají brát C soubory. Při použití cell magic stačí přidat `-I .` parametr, ten řekne cythonu, že se má koukat na zdrojáky do současné složky.

In [ ]:
%load_ext Cython

In [ ]:
%%cython -I .
cimport numpy as cnp
import numpy as np

cdef extern from "matrix_vector_multiply.c":
    void matrix_vector_multiply(float *matrix, float *vector, float *result, int rows, int cols)

def mvp_func_cy(cnp.ndarray[cnp.float32_t, ndim=2] matrix, cnp.ndarray[cnp.float32_t, ndim=1] vector, cnp.ndarray[cnp.float32_t, ndim=1] result):
    cdef int rows = matrix.shape[0]
    cdef int cols = matrix.shape[1]
    
    matrix_vector_multiply(<float*> &matrix[0, 0], <float*> &vector[0], <float*> &result[0], rows, cols)


In [ ]:
mvp_func_cy(matrix, vector, result)

print("Result:")
print(result)

Pokud bychom chtěli použít setup.py, bude to vypadat následovně:

In [ ]:
%%writefile mvp_cy.pyx

cimport numpy as cnp
import numpy as np

cdef extern from "matrix_vector_multiply.c":
    void matrix_vector_multiply(float *matrix, float *vector, float *result, int rows, int cols)

def mvp_func_cy(cnp.ndarray[cnp.float32_t, ndim=2] matrix, cnp.ndarray[cnp.float32_t, ndim=1] vector, cnp.ndarray[cnp.float32_t, ndim=1] result):
    cdef int rows = matrix.shape[0]
    cdef int cols = matrix.shape[1]
    
    matrix_vector_multiply(<float*> &matrix[0, 0], <float*> &vector[0], <float*> &result[0], rows, cols)


In [ ]:
%%writefile setup.py

from setuptools import setup, Extension
from Cython.Build import cythonize
import numpy as np

ext_modules = [
    Extension(
        "mvp_cy",
        ["mvp_cy.pyx"],
        include_dirs=[np.get_include(),"."],
        language="c",
    )
]

setup(
    name="mvp_cy",
    ext_modules=cythonize(ext_modules),
)


In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import mvp_cy

In [ ]:
mvp_cy.mvp_func_cy(matrix, vector, result)
print("Result:")
print(result)